In [ ]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle

lines = []
i = 0
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if(i > 0): #this part is added to insure the 1st line in the csvfile which include the headers is not included
             lines.append(line)
        i = i + 1

# create adjusted steering measurements for the side camera images
correction = 0.01 # this is a parameter to tune

images = []
measurments = []
for line in lines:
    #print(line)
    for i in range(3):
        # get the image path from the file
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'IMG/'+ filename
        #print('current_path', current_path)
        image = cv2.imread(current_path)
        images.append(image)
        # get the steering measurmement (labels) from the file
        measurment = line[3]
        if (i == 0):
            measurments.append(float(measurment))
        if (i == 1):
             measurments.append(float(measurment) + correction)
        if (i == 2):
             measurments.append(float(measurment) - correction)

# convert features and labels to numpy array as that's the format keras requires  
print("Sample of labels", measurments[4823] )
#X_train = np.array(images)
#y_train = np.array(measurments) 
#print(X_train[0].shape, ' shape of train samples')
#print("size of training set", len(X_train))

from sklearn.utils import shuffle
# Shuffle training data, NOTE: No need for that part as it will be done in the model.fit()
#X_train, y_train = shuffle(X_train, y_train)

# Data Augmentation
augmented_images, augmented_measurments = [], []
for image,measurment in zip(images, measurments):
    augmented_images.append(image)
    augmented_measurments.append(measurment)
    augmented_images.append(cv2.flip(image, 1)) #flip image 180 horizontally
    augmented_measurments.append(measurment * -1.0) # reverse the steering angle
   



X_train = np.array(augmented_images)
y_train = np.array(augmented_measurments) 
# display sample image and it's corresponding label (steerng angle)
print(X_train[0].shape, ' shape of train samples')
#print(" a sample of label before flipping", y_train[4823])
#print(" a sample of label after  flipping", y_train[4824])
print("size of training set", len(X_train))

#print (" Subset y-labels are")
#print(y_train[0:1000])
    


# build a very simple model to test on it
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense  
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda


model = Sequential()
#model.add(Convolution2D(32, 3, 3, input_shape=(160, 320, 3)))
# Preprocessing the data using Normalization and Mean-Center
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# Using LeNet5 model 
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, epochs=5)
#x_test = X_train
#y_test = y_train
#model.fit(X_train, y_train, batch_size=128,epochs=7,verbose=1,validation_data=(x_test, y_test))

# save the model to reuse or download
model.save('model.h5')

# NOTE: by default Keras train for 10 epochs
# you will notice that before the 7th-epoch, the validation loss was decreasing then after it increased, so to prevent overfittng, we can nb_epoch = 7

print ('end')


Sample of labels -0.3211613
(160, 320, 3)  shape of train samples
size of training set 48216


Using TensorFlow backend.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


Train on 33751 samples, validate on 14465 samples
Epoch 1/5


In [ ]:
# Load pickled data
import pickle
import numpy as np
import tensorflow as tf
tf.python.control_flow_ops = tf

with open('./small_traffic_set/small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)

X_train, y_train = data['features'], data['labels']

# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D

# Build Convolutional Neural Network in Keras Here
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

# Preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 )

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, nb_epoch=3, validation_split=0.2)

In [ ]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])